In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps

# Import linear regression from the SciPy stats module.
from scipy.stats import linregress

# Import the API key.
from config import weather_api_key
from config import gkey

# Load in csv
WeatherPy_vacation_df = pd.read_csv("weather_data/WeatherPy_vacation.csv")
WeatherPy_vacation_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Pocone,BR,68.25,clear sky,-16.26,-56.62,Hotel e Churrascaria Pantaneira
1,6,Quelimane,MZ,66.20,clear sky,-17.88,36.89,Hotel Flamingo
2,10,Bambous Virieux,MU,71.60,shower rain,-20.34,57.76,Casa Tia Villa
3,16,Quatre Cocos,MU,71.60,shower rain,-20.21,57.76,Veranda Palmar Beach Hotel
4,31,La Ronge,CA,69.80,overcast clouds,55.10,-105.28,Northland Motor Hotel
...,...,...,...,...,...,...,...,...
102,521,Gamba,GA,69.96,scattered clouds,-2.65,10.00,NaN
103,522,Maragogi,BR,74.37,overcast clouds,-9.01,-35.22,Salinas do Maragogi All Inclusive Resort
104,529,Touros,BR,73.40,broken clouds,-5.20,-35.46,INN NEW HORIZON
105,541,Cabedelo,BR,73.40,broken clouds,-6.98,-34.83,Lua Mansa Pousada


In [2]:
# choose four cities that a customer might want to visit. 
city1 = input("Choose 4 cities you would like to visit, city number 1:")
city2= input("City number 2:")
city3= input("City number 3:")
city4= input("City number 4:")

Choose 4 cities you would like to visit, city number 1:Albany
City number 2:Bethel
City number 3:Palmer
City number 4:Fitchburg


In [3]:
# Create separate DataFrames for each city on the travel route.
city1_df=WeatherPy_vacation_df.loc[(WeatherPy_vacation_df["City"]==city1)]
city1_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,33,Albany,US,71.01,clear sky,42.6,-73.97,NaN


In [4]:
city2_df=WeatherPy_vacation_df.loc[(WeatherPy_vacation_df["City"]==city2)]
city2_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,47,Bethel,US,69.01,clear sky,41.37,-73.41,Courtyard by Marriott Danbury


In [5]:
city3_df=WeatherPy_vacation_df.loc[(WeatherPy_vacation_df["City"]==city3)]
city3_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
78,397,Palmer,US,69.01,clear sky,42.16,-72.33,Wedgewood Motel


In [6]:
city4_df=WeatherPy_vacation_df.loc[(WeatherPy_vacation_df["City"]==city4)]
city4_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
18,72,Fitchburg,US,69.01,few clouds,42.58,-71.8,Days Inn by Wyndham Leominster/Fitchburg Area


In [7]:
city1_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,33,Albany,US,71.01,clear sky,42.6,-73.97,NaN


In [8]:
# Get the latitude-longitude pairs for each city DataFrame as tuples.
city1_lat=city1_df.iloc[0]["Lat"]
city1_lng=city1_df.iloc[0]["Lng"]
city1_coord=(city1_lat,city1_lng)
city1_coord

(42.6, -73.97)

In [9]:
city2_lat=city2_df.iloc[0]["Lat"]
city2_lng=city2_df.iloc[0]["Lng"]
city2_coord=(city2_lat,city2_lng)
city2_coord


(41.37, -73.41)

In [10]:
city3_lat=city3_df.iloc[0]["Lat"]
city3_lng=city3_df.iloc[0]["Lng"]
city3_coord=(city3_lat,city3_lng)
city3_coord

(42.16, -72.33)

In [11]:
city4_lat=city4_df.iloc[0]["Lat"]
city4_lng=city4_df.iloc[0]["Lng"]
city4_coord=(city4_lat,city4_lng)
city4_coord


(42.58, -71.8)

In [12]:
# Map each city on the route, travel mode - driving.
import gmaps.datasets
gmaps.configure(api_key=gkey)

city1_coord
city2_coord
city3_coord
city4_coord

fig = gmaps.figure()
city1_coord2city2_coord = gmaps.directions_layer(city1_coord,city4_coord, waypoints=[city2_coord,city3_coord],travel_mode='DRIVING')
fig.add_layer(city1_coord2city2_coord )
fig



Figure(layout=FigureLayout(height='420px'))

In [13]:
#Combine the four separate DataFrames for each city into one DataFrame.
combined_cities_df=pd.concat([city1_df,city2_df,city3_df,city4_df])
combined_cities_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,33,Albany,US,71.01,clear sky,42.60,-73.97,NaN
9,47,Bethel,US,69.01,clear sky,41.37,-73.41,Courtyard by Marriott Danbury
78,397,Palmer,US,69.01,clear sky,42.16,-72.33,Wedgewood Motel
18,72,Fitchburg,US,69.01,few clouds,42.58,-71.80,Days Inn by Wyndham Leominster/Fitchburg Area


In [14]:
# Add a pop-up marker for each city that displays the hotel name, city name, country, and maximum temperature.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

In [15]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in combined_cities_df.iterrows()]

In [16]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = combined_cities_df[["Lat", "Lng"]]
max_temp = combined_cities_df["Max Temp"]
fig = gmaps.figure(center=(42.0, -70.0), zoom_level=5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))